# Notebook Purpose

Evaluating the performance of the network when lighting is moved around in a sphere. Data is not rendered here, simply forward propagated through the network and stored for later visualization.

In [1]:
#!/usr/bin/env python
from __future__ import print_function, division
# coding: utf-8

# In[1]:


import torch
import sys

# from torchvision import datasets, models, transforms

from PIL import Image
import torchvision

sys.path.append('/om5/user/smadan/training_scaffold_own/res/')
from models.models import get_model
from loader.loader import get_loader
import random
import pickle
import argparse
import wandb

/net/storage001.ib.cluster/om2/user/smadan/differentiable_graphics_ml/training_models
/om5/user/smadan/training_scaffold_own/res/loader/multi_attribute_loader.py
/om5/user/smadan/training_scaffold_own/res/loader
/om5/user/smadan/training_scaffold_own/res/loader/loader.py
/om5/user/smadan/training_scaffold_own/res/loader


In [2]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--num_epochs', type = int, default = 50)
# parser.add_argument('--model_arch', type = str, default = 'simple_cnn')
# parser.add_argument('--batch_size', type = int, default = 100)
# parser.add_argument('--num_classes', type = int, default = 11)
# parser.add_argument('--base_lr', type = float, default = 0.001)
# parser.add_argument('--use_gpu', type = bool, default = True)
# parser.add_argument('--run_name', type = str)
# parser.add_argument('--pretrained', type = bool, default = False)

# parser.add_argument('--dataset_name', type = str, required = True)

# args = parser.parse_args()

# wandb_config = dict(vars(args))
# wandb.login()
# NUM_EPOCHS = args.num_epochs
# BATCH_SIZE = args.batch_size
# MODEL_ARCH = args.model_arch
# NUM_CLASSES = args.num_classes
# BASE_LR = args.base_lr
# USE_GPU = args.use_gpu
# SAVE_PATH = args.save_path
# LOG_FILE = args.log_file
# DATASET_NAME = args.dataset_name


# LOG_FILE_HANDLE = open(LOG_FILE, 'w')

In [4]:
wandb_config = {}
wandb_config['dataset_name'] = 'shapenet_illumination_large'
wandb_config['batch_size'] = 20

In [5]:
# In[4]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections
from collections import OrderedDict
from tqdm import tqdm

image_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
loader_new = get_loader('multi_attribute_loader_file_list_shapenet')
file_list_root = '/om5/user/smadan/dataset_lists_openmind'
att_path = '/om5/user/smadan/differentiable_graphics_ml/training_models/shapenet_id_to_class_num.p'
shuffles = {'test':False}

In [7]:
# if wandb_config['use_gpu']:
#     torch.cuda.set_device(0)

count=0

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.ToTensor()
    ]),
}

file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}
for phase in ['test']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,wandb_config['dataset_name'])
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform)
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=wandb_config['batch_size'], shuffle = shuffles[phase], num_workers=2,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])

In [8]:
loaded_model = torch.load('/om5/user/smadan/differentiable_graphics_ml/training_models/saved_models/alexnet_10_models_00001_3_final.pt')

In [9]:
light_point_to_matches = {}

for data in tqdm(dset_loaders['test']):
    inputs, labels, image_paths = data
    inputs = inputs.float().cuda()
    labels = labels.long().cuda()

    outputs = loaded_model(inputs)
    _, preds = torch.max(outputs.data, 1)
    for i in range(wandb_config['batch_size']):
        image_name = image_paths[i].split('/')[-1]
        category, instance, light_x, light_y, light_z = image_name.split('_')
        light_z = light_z.split('.png')[0]
        light_point = "%s_%s_%s"%(light_x, light_y, light_z)
        match = labels[i] == preds[i]
        if light_point in light_point_to_matches.keys():
            light_point_to_matches[light_point].append(match.item())
        else:
            light_point_to_matches[light_point] = [match.item()]

100%|██████████| 648/648 [00:36<00:00, 17.76it/s]


In [10]:
light_point_to_accuracies = {}

In [11]:
for light_point in light_point_to_matches.keys():
    matches = light_point_to_matches[light_point]
    accuracy = torch.sum(torch.tensor(matches))/float(len(matches))
    light_point_to_accuracies[light_point] = accuracy

In [12]:
with open('light_point_to_accuracies_large.p','wb') as F:
    pickle.dump(light_point_to_accuracies,F)